In [30]:
import numpy as np
import torch
from deepymod_torch.library_function import library_1D_in
from deepymod_torch.neural_net import deepmod_init, train
from deepymod_torch.sparsity import scaling, threshold

torch.set_default_tensor_type('torch.cuda.FloatTensor')  # enable for gpu.

np.random.seed(42)

%load_ext autoreload
%autoreload 2

In [31]:
number_of_samples = 1000

data = np.load('../data/processed/burgers.npy', allow_pickle=True).item()

X = np.transpose((data['t'].flatten(), data['x'].flatten()))
y = np.real(data['u']).reshape((data['u'].size, 1))

idx = np.random.permutation(y.size)
X_train = torch.tensor(X[idx, :][:number_of_samples], dtype=torch.float32, requires_grad=True)
y_train = torch.tensor(y[idx, :][:number_of_samples], dtype=torch.float32)

data = X_train
target = y_train

In [32]:
optim_config = {'lambda': 10**-6, 'max_iterations': 1000}
library_config = {'type': library_1D_in, 'poly_order': 1, 'diff_order': 2}
network_config = {'input_dim': 2, 'hidden_dim': 20, 'layers': 5, 'output_dim': 1}

In [33]:
# Initiating
network, coeff_vector, sparsity_mask = deepmod_init(network_config, library_config)

In [34]:
coeff_vector

tensor([[ 0.0830],
        [ 0.9844],
        [ 0.6350],
        [-0.4638],
        [ 0.8440],
        [ 0.8816]], requires_grad=True)

In [35]:
sparsity_mask

tensor([0, 1, 2, 3, 4, 5])

In [36]:
time_deriv, theta, coeff_vector = train(data, target, network, coeff_vector, sparsity_mask, library_config, optim_config)

Epoch | Total loss | MSE | PI | L1 
0 3.6E-02 3.6E-02 1.5E-04 5.0E-05
tensor([[ 0.0810],
        [ 0.9824],
        [ 0.6330],
        [-0.4658],
        [ 0.8421],
        [ 0.8798]], requires_grad=True)
500 7.7E-04 5.2E-04 2.4E-04 1.3E-06
tensor([[ 0.0040],
        [-0.0653],
        [ 0.0523],
        [-0.0434],
        [-0.3045],
        [ 0.0325]], requires_grad=True)


In [37]:
coeff_vector

tensor([[ 0.0022],
        [-0.1777],
        [ 0.0482],
        [-0.0081],
        [-0.4179],
        [ 0.0877]], requires_grad=True)

In [38]:
scaled_coeff_vector = scaling(coeff_vector, theta, time_deriv)

In [43]:
sparse_coeff_vector, sparsity_mask = threshold(scaled_coeff_vector, coeff_vector)
print(coeff_vector, sparse_coeff_vector, sparsity_mask)

tensor([[ 0.0022],
        [-0.1777],
        [ 0.0482],
        [-0.0081],
        [-0.4179],
        [ 0.0877]], requires_grad=True) tensor([[-0.1777],
        [-0.4179]], requires_grad=True) tensor([1, 4])


In [44]:
sparsity_mask.requires_grad

False

In [45]:
time_deriv, theta, coeff_vector = train(data, target, network, sparse_coeff_vector, sparsity_mask, library_config, optim_config)

Epoch | Total loss | MSE | PI | L1 
0 8.8E-04 4.6E-04 4.2E-04 1.0E-06
tensor([[-0.1757],
        [-0.4159]], requires_grad=True)
500 4.2E-04 2.0E-04 2.2E-04 9.6E-07
tensor([[-0.2579],
        [-0.1660]], requires_grad=True)


In [ ]:
sparse_coeff_vector, sparsity_mask, network = DeepMoD(X_train, y_train, network_config, lib_config, optim_config)

print(sparse_coeff_vector)

In [ ]:
def 